# 030 Get sustainable CDR range from C1- and C2-type pathways 

The raw data is from IPCC AR6 WGIII (Riahi et al., 2022; Byers et al., 2022).

References:

> Riahi, K. et al. Mitigation pathways compatible with long-term goals. in IPCC, 2022: Climate Change 2022: Mitigation of Climate Change. Contribution of Working Group III to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change (eds. Shukla, P. R. et al.) (Cambridge University Press, 2022). doi:10.1017/9781009157926.005.

> Byers, E. et al. AR6 Scenarios Database. (2022) doi:10.5281/zenodo.5886912.

In [1]:
import pyam
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import scmdata

import utils

<IPython.core.display.Javascript object>

# Filter C1 and C2-scenarios from large AR6 database

## Read in data

In [26]:
data_folder = Path('../data/raw/ar6-wg3/')
data_file = Path(data_folder / 'AR6_Scenarios_Database_World_v1.1.csv')
meta_file = Path(data_folder / 'AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx')

output_folder = Path('../data/interim/ar6-wg3-cdr/')

In [27]:
df_ar6=pyam.IamDataFrame(
    data_file
)

pyam.core - INFO: Reading file ../data/raw/ar6-wg3/AR6_Scenarios_Database_World_v1.1.csv


In [28]:
# Metadata for the scenarios
df_ar6.load_meta(
    meta_file,
    sheet_name='meta_Ch3vetted_withclimate'
)

pyam.core - INFO: Reading meta indicators for 1202 out of 2298 scenarios


In [29]:
df_ar6.filter(
    Category=['C2'],
    inplace=True
)

In [30]:
df_ar6.meta.Policy_category_name.unique()

array(['P2a: Immediate global action without transfers',
       'P2: Immediate global action', 'P3b: NDC + Delayed global action',
       'P2c: Other policies + Immediate global action',
       'P2b: Immediate global action with transfers',
       'P0_2 Diagnostic around an immediate action case',
       'P3c: Other policies + Delayed global action'], dtype=object)

In [31]:
df_ar6.to_csv(
    Path(output_folder / '0_ar6_C1C2.csv')
)

# Process CDR data
### Set up net-negative AFOLU variable

In [32]:
df_ar6_afolu = df_ar6.filter(
    variable='Emissions|CO2|AFOLU'
)

Interpolate for missing 5 year steps

In [33]:
df_ar6_afolu.interpolate(
    time=range(2010,2105,5),
    inplace=True
)

In [34]:
df_ar6_afolu_ts = df_ar6_afolu.timeseries()

In [35]:
df_afolu_nn = (
    df_ar6_afolu_ts
    .applymap(lambda x: max(0, -x))
    .reset_index()
)

In [36]:
df_afolu_nn['variable'] = 'Carbon Sequestration|AFOLU|Net Negative'

In [37]:
df_ar6.append(
    pyam.IamDataFrame(
        df_afolu_nn
    ),
    inplace = True
)

### Correct incorrect sign for some Remind DAC trajectories

In [38]:
df_remind_dac = df_ar6.filter(
    model = 'REMIND*',
    scenario = 'EN*',
    variable = 'Carbon Sequestration|Direct Air Capture'
)

In [39]:
df_remind_dac.convert_unit(
    current = 'Mt CO2/yr',
    to = 'Mt CO2/yr',
    factor = -1,
    inplace = True
)

replace in original data

In [40]:
df_ar6.filter(
    model = 'REMIND*',
    scenario = 'EN*',
    variable = 'Carbon Sequestration|Direct Air Capture',
    keep = False,
    inplace = True
)

In [41]:
df_ar6.append(
    df_remind_dac,
    inplace = True
)

In [42]:
df_ar6_cdr=df_ar6.filter(
    variable=[ 
        'Carbon Sequestration|CCS|Biomass',
        'Carbon Sequestration|AFOLU|Net Negative',
        'Carbon Sequestration|Direct Air Capture',
        'Carbon Sequestration|Enhanced Weathering'
    ]
)

In [43]:
df_ar6_cdr.interpolate(
    time=range(2010, 2105,5),
    inplace=True
)

### Create a CDR variable

In [44]:
df_ar6_total_cdr = df_ar6_cdr.aggregate(
    variable='Carbon Dioxide Removal',
    components=[
        'Carbon Sequestration|CCS|Biomass',
        'Carbon Sequestration|AFOLU|Net Negative',
        'Carbon Sequestration|Direct Air Capture',
        'Carbon Sequestration|Enhanced Weathering'
    ]
)

### Save to file

In [45]:
df_ar6_total_cdr.to_csv(
    Path(output_folder / '1_ar6_cdr.csv')
)

In [46]:
df_ar6.append(
    pyam.IamDataFrame(
        df_ar6_total_cdr
    ),
    inplace=True,
    ignore_meta_conflict=True
)

# Set 2100 CDR values to meta column

In [47]:
df_ar6.set_meta_from_data(
    name='cdr_2100',
    variable='Carbon Dioxide Removal',
    year=2100
)

In [48]:
df_ar6.set_meta_from_data(
    name='emi_co2_2100',
    variable='AR6 climate diagnostics|Infilled|Emissions|CO2',
    year=2100
)

In [49]:
df_ar6.meta[['cdr_2100', 'emi_co2_2100']].describe()

,cdr_2100,emi_co2_2100
count,133.000000,133.000000
mean,14321.489202,-12303.154476
std,7074.721761,5021.422197
min,-6517.954800,-21722.688129
25%,11753.264275,-15782.027678
50%,16193.945667,-12430.507194
75%,19044.920097,-9608.804982
max,28776.865479,-295.682656


#### C1 & C2 pathway: median 14900, max 28800
#### C1 pathway only: median 11800, max 20900
#### C2 pathway only: median 16200, max 28800